In [1]:
import os
import torch
import huggingface_hub

import json
import pandas as pd

from langchain.vectorstores import FAISS  # FAISS로 변경
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.schema.runnable import RunnableLambda
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
# 문서 데이터 로드
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
        
doc_file = "../dataset/documents.jsonl"
doc_data = load_jsonl(doc_file)
print(len(doc_data))

4272


In [3]:
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {"device": "cuda:0"}
encode_kwargs = {"normalize_embeddings": False}

embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
contents = [doc['content'] for doc in doc_data] ## 벡터 DB 생성 및 임베딩 벡터 저장

vector_db = FAISS.from_texts(texts=contents, embedding=embedding_model)
retriever = vector_db.as_retriever(search_kwargs={"k": 10})  ## 상위 10개 검색

## 벡터 DB에 저장된 벡터의 수를 확인
print(f"벡터 DB에 저장된 벡터의 수: {vector_db.index.ntotal}")
print(f"입력한 텍스트의 수: {len(contents)}")

/home/pervinco/miniconda3/envs/IR/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


벡터 DB에 저장된 벡터의 수: 4272
입력한 텍스트의 수: 4272


In [4]:
# 임의의 쿼리로 벡터 DB에서 검색
query = "에너지 균형을 유지하는 방법은 무엇인가요?"
results = retriever.get_relevant_documents(query)

# 검색 결과 출력
for idx, result in enumerate(results):
    print(f"검색 결과 {idx + 1}: {result.page_content}")

검색 결과 1: 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.
검색 결과 2: 에너지 전달은 다양한 형태와 방식으로 이루어집니다. 에너지는 한 형태에서 다른 형태로 전달될 수 있으며, 이는 우리 일상 생활에서도 많이 경험할 수 있습니다. 예를 들어, 태양에서 나오는 에너지는 태양광 전지를 통해 전기 에너지로 변환될 수 있습니다. 또한, 운동 에너지는 자전거의 페달을 밟으면서 전기 에너지로 변환될 수 있습니다. 이처럼 에너지 전달은 다양한 종류와 방식을 가지고 있습니다.

하지만, 모든 종류의 에너지 전달을 가장 알맞게 설명하는 명제는 '사용할 수 있는 에너지의 감소를 초래합니다.'입니다. 에너지는 전달되는 과정에서 일부가 손실되기 때문에, 전달된 에너지의 양은 원래의 양보다 적어집니다. 이러한 손실은 에너지의 효율성을 나타내는 중요한 요소 중 하나입니다. 에너지 전달 과정에서 발생하는 손실을 최소화하기 위해 우리는 다양한 기술과 방법을 개발하고 연구하고 있습니다.

에너지 전달은 우리의 삶과 사회에 매우 중요한 역할을 합니다. 우리는 에너지를 사용하여 가정이나 사무실을 데워주고, 차량을 움직이게 하며, 전자기기를 작동시킵니다. 따라서, 에너지 전달에 대한 이해와 연구는 우리의 삶의 질을 향상시키는 데에 큰 도움이 될 것입니다.
검색 결

/tmp/ipykernel_939424/3692756873.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [5]:
## LLaMa 3.1 모델 로드
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    else:
        tokenizer.pad_token = tokenizer.eos_token = '<|endoftext|>' 

## Tokenizer와 Model을 LangChain에 연동
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=320,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    clean_up_tokenization_spaces=True  # 또는 False로 설정 가능
)

hf = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str):
        split_text = text.split("도움이 되는 답변: ", 1)
        if len(split_text) > 1:
            return split_text[1].strip()
        else:
            return text

def docs_to_text(docs):
    return "\n\n".join([doc.page_content for doc in docs[:3]])  # 상위 3개 문서 사용


# 프롬프트 템플릿 수정
template = """다음과 같은 맥락을 사용해서 마지막 질문에 대답하세요.
{context}
질문: {question}
도움이 되는 답변: """

rag_prompt = PromptTemplate.from_template(template)

output_parser = CustomOutputParser()
top3_context = RunnableLambda(docs_to_text)

In [7]:
rag_chain = {
    "context": retriever | top3_context,
    "question": RunnablePassthrough()
} | rag_prompt | hf | output_parser

# Chain 실행
response = rag_chain.invoke("건강한 사람이 에너지 균형을 평형 상태로 유지하는 것이 중요해??")
print(response)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


1-2주의 기간 동안 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 건강한 사람은 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 매우 중요합니다. 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 하며, 이를 통해 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 건강한 사람이 에너지 균형을 유지하는 것은 매우 중요합니다. 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 하며, 이를 통해 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 건강한 사람이 에너지 균형을 유지하는 것은 매우 중요합니다. 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 하며, 이를 통해 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 건강한 사람이 에너지 균형을 유지하는 것은 매우 중요합니다. 올바른 식단과 적절한 운동을 통해 에너지 섭
